In [1]:
import os
import numpy as np
import matplotlib
import pandas as pd
from matplotlib import pyplot as plt
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
import seaborn as sns
sns.set_style('white')
import dataframe_image as dfi
import pickle
from network_setup import *
from data_setup import *

from EnergyCommunityModel import *
from model_eval import *

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
# set up simulation scope
input_cdir_timestamp = '2022 11 09 10h46m56s'
basepath = os.path.join(os.getcwd(), '02_implementation\energy_model')
with open(os.path.join(basepath,'2022_11_13_filepaths.pickle'), 'rb') as handle:
    paths = pickle.load(handle)
date_t = '2022 12 16 20h00m'
n_steps = 96*7

# define comprehensive scenario settings for analysis
scenario_sc0 = {'seasons': ['summer', 'winter'], 'p2p_options': [False], 'pr_years': [2019, 2022], 'pr_dynamic_options': [False]}
scenario_sc1 = {'seasons': ['summer', 'winter'], 'p2p_options': [False], 'pr_years': [2019, 2022], 'pr_dynamic_options': [True]}
scenario_sc2 = {'seasons': ['summer', 'winter'], 'p2p_options': [True], 'pr_years': [2019, 2022], 'pr_dynamic_options': [True]}
scenario_dict = {'scenario_0': scenario_sc0, 'scenario_1': scenario_sc1, 'scenario_2': scenario_sc2}

# manually define all scenarios
scenario_0 = {'seasons': ['summer'], 'p2p_options': [False], 'pr_years': [2019], 'pr_dynamic_options': [False]}
scenario_1 = {'seasons': ['winter'], 'p2p_options': [False], 'pr_years': [2019], 'pr_dynamic_options': [False]}
scenario_2 = {'seasons': ['summer'], 'p2p_options': [False], 'pr_years': [2022], 'pr_dynamic_options': [False]}
scenario_3 = {'seasons': ['winter'], 'p2p_options': [False], 'pr_years': [2022], 'pr_dynamic_options': [False]}

scenario_4 = {'seasons': ['summer'], 'p2p_options': [False], 'pr_years': [2019], 'pr_dynamic_options': [True]}
scenario_5 = {'seasons': ['winter'], 'p2p_options': [False], 'pr_years': [2019], 'pr_dynamic_options': [True]}
scenario_6 = {'seasons': ['summer'], 'p2p_options': [False], 'pr_years': [2022], 'pr_dynamic_options': [True]}
scenario_7 = {'seasons': ['winter'], 'p2p_options': [False], 'pr_years': [2022], 'pr_dynamic_options': [True]}

scenario_8 = {'seasons': ['summer'], 'p2p_options': [True], 'pr_years': [2019], 'pr_dynamic_options': [True]}
scenario_9 = {'seasons': ['winter'], 'p2p_options': [True], 'pr_years': [2019], 'pr_dynamic_options': [True]}
scenario_10 = {'seasons': ['summer'], 'p2p_options': [True], 'pr_years': [2022], 'pr_dynamic_options': [True]}
scenario_11 = {'seasons': ['winter'], 'p2p_options': [True], 'pr_years': [2022], 'pr_dynamic_options': [True]}

all_scs = [scenario_0, scenario_1, scenario_2, scenario_3, scenario_4, scenario_5, 
           scenario_6, scenario_7, scenario_8, scenario_9, scenario_10, scenario_11]

---

#### Plotting P2P price levels

In [ ]:
# read in mdf...

In [ ]:
# plot p2p price level
l = mdf.price_n_level.tolist()
ind = mdf.index
dfs = []
for i in range(6):
    dfs.append(pd.DataFrame([j[i] for j in l], index=ind))
df = pd.concat(dfs, axis=1)
df.columns = ['price g_s', 'price_n_d (1)', 'price_n_d (2)', 'price_n_d (3)', 'price_n_d (4)', 'price g_d']

fig, ax = plt.subplots(figsize = (22,6))
df.plot(ax = ax, title='P2P trading price level (ct/kWh), summer 2022', color = ['blue', 'grey', 'grey','grey','grey', 'black'])
ax.set_ylabel('ct/kWh')
plt.tight_layout()
plt.show()

---

#### Get overall stability metrics

In [ ]:
mdf = get_quick_mdf(paths, date_t, scenario_dict_sc=scenario_11)
df = calculate_stability_measures(mdf)
df

,net_grid_demand,autarky_level
count,672.00,672.00
mean,-1.01,2.82
std,5.17,8.65
max,31.13,50.81
sum,-680.81,0.00


In [ ]:
# calculate certain measure for certain scenario and save df as png
m_dict = calculate_measure(paths, date_t, scenario_dict=scenario_2, measure='welfare')
#export_measure_calculation(paths, m_dict)

---

#### Benchmark: Effectiveness of P2P trading heuristic
Compare total energy costs with and without p2p trading heuristic

In [3]:
# baseline: with p2p trading heuristic
date_t = '2022 12 16 20h00m'
df = get_quick_measure_kpis(paths, date_t, scenarios=all_scs, measure='welfare')
df

,year,season,price,p2p,use case,total_costs_all,total_costs_prosumer,total_costs_consumer,mean_gini
sc0,2019,summer,static,disabled,"uc_1: static prices, no p2p trading",-9701.835946,-22470.212275,12768.376329,0.676686
sc1,2019,winter,static,disabled,"uc_1: static prices, no p2p trading",6401.685978,-6654.980249,13056.666227,0.567819
sc2,2022,summer,static,disabled,"uc_1: static prices, no p2p trading",853.982288,-22446.615613,23300.597901,0.679167
sc3,2022,winter,static,disabled,"uc_1: static prices, no p2p trading",14162.493037,-5491.606116,19654.099152,0.555202
sc4,2019,summer,dynamic,disabled,"uc_2: dynamic prices, no p2p trading",-10407.758106,-22476.353013,12068.594907,0.656174
sc5,2019,winter,dynamic,disabled,"uc_2: dynamic prices, no p2p trading",5617.29523,-7046.423116,12663.718346,0.558126
sc6,2022,summer,dynamic,disabled,"uc_2: dynamic prices, no p2p trading",-676.368955,-22448.518689,21772.149734,0.651378
sc7,2022,winter,dynamic,disabled,"uc_2: dynamic prices, no p2p trading",11947.305781,-6147.817444,18095.123225,0.552212
sc8,2019,summer,dynamic,enabled,"uc_3: dynamic prices, p2p trading",-12292.010481,-24307.679507,12015.669025,0.656489
sc9,2019,winter,dynamic,enabled,"uc_3: dynamic prices, p2p trading",4459.963356,-8142.977327,12602.940684,0.562082


In [7]:
# without p2p trading heuristic
date_t = '2023 03 07 16h00m'

df = calculate_measure(paths, date_t, scenario_dict=scenario_11, to_df=True, measure='welfare')
df

,costs_all,costs_prosumer,costs_consumer,costs_gini
count,10.00,10.00,10.00,10.00
mean,26.66,12.73,13.94,0.36
std,12.05,6.41,6.34,0.08
min,16.50,7.27,8.84,0.25
max,53.51,24.90,28.61,0.44
sum,266.63,127.27,139.36,0.00


In [8]:
# without p2p trading heuristic
date_t = '2023 03 07 17h00m'

df = calculate_measure(paths, date_t, scenario_dict=scenario_11, to_df=True, measure='welfare')
df

,costs_all,costs_prosumer,costs_consumer,costs_gini
count,10.00,10.00,10.00,10.00
mean,26.66,12.73,13.94,0.36
std,12.05,6.41,6.34,0.08
min,16.50,7.27,8.84,0.25
max,53.51,24.90,28.61,0.44
sum,266.63,127.27,139.36,0.00
